In [ ]:
## We need city csv; we need places shp file for the FIPS

In [7]:
import pandas as pd
import geopandas as gpd
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [2]:
# load city info used in Heat Pump Calculator (HPC)
city_url = 'https://github.com/leungkp/akwlib-export/blob/main/data/v01/city.csv' + '?raw=true'
city_HPC = pd.read_csv(city_url)

In [3]:
city_HPC.head()

,ID,Name,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,Oil1Price,...,TMYname,ElecUtilities,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage
0,1,Anchorage,61.151960,-149.864100,2,2,1,0,NaN,4.45,...,"ANCHORAGE INTL AP, AK","[('Chugach Electric South- Residential', 1), (...",0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815..."
1,2,Adak,51.848896,-176.635422,2,3,5,0,NaN,7.30,...,"ADAK NAS, AK","[('Adak -TDX Residential', 677), ('Adak - TDX ...",NaN,Adak,Adak city,Aleutians West Census Area,Aleut,Affordable Energy Strategy Area,False,"[471.936998147, 423.45531781, 460.81549341, 42..."
2,3,Akutan,54.134945,-165.772446,2,3,5,0,NaN,4.93,...,"DUTCH HARBOR, AK","[('Akutan, City of - Residential', 9), ('Akuta...",NaN,Akutan,Akutan city,Aleutians East Borough,Aleut,Affordable Energy Strategy Area,False,"[500.99156233, 439.984315846, 456.510655517, 4..."
3,4,Allakaket,66.561386,-152.649445,4,4,5,0,NaN,7.00,...,"BETTLES FIELD, AK","[('Allakaket - APT - Residential', 249), ('All...",NaN,Allakaket,Allakaket city,Yukon-Koyukuk Census Area,Doyon,Affordable Energy Strategy Area,False,"[338.509000744, 291.889556239, 276.804469551, ..."
4,5,Angoon,57.496647,-134.578900,1,1,3,0,NaN,6.20,...,"SITKA JAPONSKI AP, AK",[('Inside Passage Electric Coop - Residential'...,NaN,Angoon,Angoon city,Hoonah-Angoon Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[277.080558981, 257.757954545, 237.15561277, 2..."


In [21]:
# download and unzip blocks manually because of its big size
zipurl = 'https://live.laborstats.alaska.gov/cen/maps/gis/akplaces_2010.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('/tmp/Places2010')

In [22]:
places = gpd.read_file('/tmp/Places2010/')

In [23]:
places.head()

,NAME,STATE,PLACE,FIPS,TOTALPOP,WHITE,BLACK,NATIVE,ASIAN,PACISLAND,...,NATALNCOMB,MALE,FEMALE,GRPQTRS,HOUSEUNITS,VACANT,OCCUPIED,OWNED,RENTED,geometry
0,Sand Point city,02,67020,0267020,976.0,166.0,24.0,381.0,339.0,2.0,...,417.0,610.0,366.0,350.0,290.0,44.0,246.0,146.0,100.0,"POLYGON ((-416449.268 607136.253, -416493.560 ..."
1,King Cove city,02,39410,0239410,938.0,152.0,9.0,360.0,342.0,2.0,...,384.0,577.0,361.0,438.0,229.0,48.0,181.0,96.0,85.0,"POLYGON ((-529159.655 591264.557, -529691.096 ..."
2,False Pass city,02,24660,0224660,35.0,7.0,0.0,27.0,1.0,0.0,...,27.0,19.0,16.0,0.0,40.0,25.0,15.0,8.0,7.0,"POLYGON ((-596242.806 575598.623, -595913.325 ..."
3,Cold Bay city,02,16530,0216530,108.0,80.0,2.0,13.0,2.0,0.0,...,20.0,66.0,42.0,1.0,82.0,36.0,46.0,5.0,41.0,"POLYGON ((-551112.137 619740.119, -551194.636 ..."
4,Akutan city,02,01090,0201090,1027.0,239.0,184.0,56.0,445.0,15.0,...,76.0,790.0,237.0,937.0,44.0,4.0,40.0,13.0,27.0,"POLYGON ((-763352.775 524054.740, -763658.745 ..."


In [24]:
## match city with places (FIPS)
city_HPC['name2match'] = city_HPC['census_city'].str.lower()
places['name2match'] = places['NAME'].str.lower()

In [25]:
city_HPC['name2match'].isin(places['name2match']).mean()

1.0

In [35]:
places2match = places[['name2match', 'FIPS']]
merged_data = pd.merge(city_HPC, places2match, on='name2match', how='left')
merged_data.head()

,ID,Name,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,FuelCityID,Oil1Price,...,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match,FIPS
0,1,Anchorage,61.151960,-149.864100,2,2,1,0,NaN,4.45,...,0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
1,2,Adak,51.848896,-176.635422,2,3,5,0,NaN,7.30,...,NaN,Adak,Adak city,Aleutians West Census Area,Aleut,Affordable Energy Strategy Area,False,"[471.936998147, 423.45531781, 460.81549341, 42...",adak city,0200065
2,3,Akutan,54.134945,-165.772446,2,3,5,0,NaN,4.93,...,NaN,Akutan,Akutan city,Aleutians East Borough,Aleut,Affordable Energy Strategy Area,False,"[500.99156233, 439.984315846, 456.510655517, 4...",akutan city,0201090
3,4,Allakaket,66.561386,-152.649445,4,4,5,0,NaN,7.00,...,NaN,Allakaket,Allakaket city,Yukon-Koyukuk Census Area,Doyon,Affordable Energy Strategy Area,False,"[338.509000744, 291.889556239, 276.804469551, ...",allakaket city,0201860
4,5,Angoon,57.496647,-134.578900,1,1,3,0,NaN,6.20,...,NaN,Angoon,Angoon city,Hoonah-Angoon Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[277.080558981, 257.757954545, 237.15561277, 2...",angoon city,0203440


In [46]:
merged_data['FIPS'] = pd.to_numeric(merged_data['FIPS'])

In [51]:
merged_data.columns.tolist()

['ID',
 'Name',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYid',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match',
 'FIPS']

In [53]:
cols = ['ID',
 'Name',
 'FIPS',
 'TMYid',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match'
 ]

In [55]:
merged_data = merged_data[cols]

In [59]:
merged_data.to_csv('data/v01/city.csv', index=False)
merged_data.to_pickle('data/v01/city.pkl')

In [60]:
## amend the meta file
meta_file = pd.read_csv('data/v01/ERA5/ERA5_meta.csv')

In [73]:
merged_data2 = merged_data[['FIPS', 'TMYid']].rename(columns={'TMYid': 'tmy_id'})

In [74]:
merged_data2.head()

,FIPS,tmy_id
0,203000,702730
1,200065,704540
2,201090,704890
3,201860,701740
4,203440,703710


In [75]:
meta_file_matched = meta_file.merge(merged_data2, on='tmy_id', how='left')

In [79]:
meta_file_matched['FIPS'] = pd.to_numeric(meta_file_matched['FIPS'])

In [82]:
meta_file_matched.to_csv('data/v01/ERA5/ERA5_meta.csv', index=False)
meta_file_matched.to_pickle('data/v01/ERA5/ERA5_meta.pkl')